# TCC - Victor Gonçalves
## Instruções de uso do código
Autor: Prof. Dr. Reginaldo Gonçalves Leão Junior

### Descrição da Proposta e Ferramentas
A corrente proposta do trabalho consiste no uso de uma rede neural recorrente (RNN) com camadas do tipo *long short-term memory* para predição da potência solar na microrregião de Formiga ou na Mesorregião Oeste de Minas Gerais. Os dados de treinamento do modelo devem ser obtidos de *datasets* gratuítos disponibilizados na web e os dados climáticos da região obtidos do INMET.(https://portal.inmet.gov.br/dadoshistoricos).

Por motivos de desempenho, o modelo será desenvolvido na linguagem Julia (+1.7 - https://julialang.org/downloads/) utilizando a biblioteca Flux.jl (https://fluxml.ai/Flux.jl/stable/) para a implementação das soluções baseadas em ML. Todas as funcionalidades estão contidas em um módulo denominado `SolarPowerForecastingIFMG` desenvolvido no contexto deste trabalho de conclusão de curso.

A rotina de uso do modelo deve seguir a seguinte rotina, obviamente, flexibilizações podem ser introduzidas quando necessário, ou mesmo novas rotinas podem ser criadas.

### Verificação do status INMET

Consiste na chamada à função `statusINMET()` que verifica a disponibilidade de séries anuais climatológicas no recurso `/dadoshistoricos` do portal, exibe um compilado deste estatus no terminal e retorna um dicionário com a estrutura:

```
Dict{Int16, String} with n entries:
  "20XX" => "https://portal.inmet.gov.br/uploads/dadoshistoricos/20XX.zip"
```

no qual as chaves são os anos disponíveis na base no formato de um inteiro de 32 bits e os valores uma String contendo o link para download do zip file das séries para o respectivo ano.

Antes de qualquer verificação, deve-se fazer a instalação do pacote em desenvolvimento, ou verificação por atualizações.
Sendo o primeiro uso do pacote em uma determinada instalção deve-se executar as instruções do comentário `i)`, nas demais, atualizações devem ser checadas por meio das instruções do comentário `ii)`. A instrução `using Pkg` é apenas uma chamada ao módulo `Pkg` responsável pela gerência dos pacotes.

In [8]:
using Pkg
#= i) Na primeira execução do pacote
Pkg.develop("SolarPowerForecastingIFMG")
=#

# #= ii) A cada nova adição de funcionalidade
Pkg.update("SolarPowerForecastingIFMG")
# =#

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


Após a instalação ou atualização o pacote deve ser carregado no *kernel* em execução, em seguida a verificação do status no INMET pode ser imediatamente realizada.

In [9]:
using SolarPowerForecastingIFMG: statusINMET, obter_dados
fonte_dados = statusINMET();

Estrato de dados INMET
Primeiro ano disponível	->	2000
Último ano disponível	->	2021


┌ Info: Precompiling SolarPowerForecastingIFMG [324c0b22-a149-4aa4-8e58-0d2d69aac616]
└ @ Base loading.jl:1423
┌ Warning: Package SolarPowerForecastingIFMG does not have HTTP in its dependencies:
│ - If you have SolarPowerForecastingIFMG checked out for development and have
│   added HTTP as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with SolarPowerForecastingIFMG
└ Loading HTTP into SolarPowerForecastingIFMG from project dependency, future warnings for SolarPowerForecastingIFMG are suppressed.


Após a execução a variável `fonte_dados` é um dicionário que contém os links para a obtenção das tabelas CSV de cada ano.

In [10]:
@show fonte_dados[2000];
@show fonte_dados[2010];
@show fonte_dados[2021];

fonte_dados[2000] = "https://portal.inmet.gov.br/uploads/dadoshistoricos/2000.zip"
fonte_dados[2010] = "https://portal.inmet.gov.br/uploads/dadoshistoricos/2010.zip"
fonte_dados[2021] = "https://portal.inmet.gov.br/uploads/dadoshistoricos/2021.zip"


### Raspagem de Dados
A extração de dados em si consiste na 